In [1]:
from time import time, strftime, gmtime
import os
import numpy as np
from glob import glob
from random import choice
from pandas import DataFrame
from psychopy import visual, core, event
from pylsl import StreamInfo, StreamOutlet, local_clock
from muselsl import record
from multiprocessing import Process

from experiments.stimulus_presentation import generate_N170

ImportError: No module named 'experiments'

# N170

This notebook will run an N170 experiment to analyze potentials related to face detection.

A series of faces and houses will be presented. The task is to mentally note whether a "face" or a "house" was presentedm. 

In [2]:
# Modify these parameters 
duration = 120
subject = '3'
session = '3'

In [21]:
def generate_N170(duration):
    info = StreamInfo('Markers', 'Markers', 1, 0, 'int32', 'myuidw43536')
    outlet = StreamOutlet(info)

    markernames = [1, 2]
    start = time()

    # Set up trial parameters
    n_trials = 2010
    iti = 0.3
    soa = 0.2
    jitter = 0.2
    record_duration = np.float32(duration)

    # Setup trial list
    image_type = np.random.binomial(1, 0.5, n_trials)
    trials = DataFrame(dict(image_type=image_type,
                            timestamp=np.zeros(n_trials)))


    # Setup graphics
    def load_image(filename):
        return visual.ImageStim(win=mywin, image=filename)


    mywin = visual.Window([1600, 900], monitor='testMonitor', units='deg', winType='pygame',
                          fullscr=True)
    faces = list(map(load_image, glob(
        os.path.join(os.path.expanduser("~"), "eeg-notebooks", 'stimulus_presentation/stim/face_house/faces/*_3.jpg'))))
    houses = list(map(load_image, glob(
        os.path.join(os.path.expanduser("~"), "eeg-notebooks", 'stimulus_presentation/stim/face_house/houses/*.3.jpg'))))

    for ii, trial in trials.iterrows():
        # Intertrial interval
        core.wait(iti + np.random.rand() * jitter)

        # Select and display image
        label = trials['image_type'].iloc[ii]
        image = choice(faces if label == 1 else houses)
        image.draw()

        # Send marker
        timestamp = local_clock()
        outlet.push_sample([markernames[label]], timestamp)
        mywin.flip()

        # offset
        core.wait(soa)
        mywin.flip()
        if len(event.getKeys()) > 0 or (time() - start) > record_duration:
            break
        event.clearEvents()

    # Cleanup
    mywin.close()


In [3]:
recording_path = os.path.join(os.path.expanduser("~"), "eeg-notebooks", "data", "visual", "N170", "subject" + subject, "session" + session, ("recording_%s.csv" %
                                              strftime("%Y-%m-%d-%H.%M.%S", gmtime())) + ".csv")

stimulus = Process(target=generate_N170, args=(duration,))
recording = Process(target=record, args=(duration, recording_path))

stimulus.start()
recording.start()

NameError: name 'generate_N170' is not defined